In [47]:
import cv2
import numpy as np
from detection2 import Yaw


# name = "IMG_3603"
# name = "IMG_3468"

# name = "IMG_4295"
# name = "IMG_4296"
# name = "IMG_4297"
# name = "IMG_4299"
# name = "IMG_4300"
# name = "IMG_4301"
# name = "IMG_4302"

# name = "IMG_4384"
# name = "IMG_4385"
# name = "IMG_4386"
name = "IMG_4387"


FRAME_PER_SECOND = 4


cap = cv2.VideoCapture('sh/mov/'+ name +'.MOV')

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps    = cap.get(cv2.CAP_PROP_FPS)

shoter_time = int(fps // FRAME_PER_SECOND)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps    = cap.get(cv2.CAP_PROP_FPS)
frames = []
angles = []
yaw_obj = Yaw()


c= 1
while True:
    ret, frame = cap.read()
    if not ret:
        break

    c+=1

    yaww = yaw_obj.predict_yaw(frame)
    angles.append(yaww)
    frames.append(frame)

    if (c% 100 ==0):
        print(c , yaww)

    
cap.release()
video_array = np.array(frames)

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

# real-time

In [48]:
def find_nearest(lst, num):
    if num is  None:
        return None

    nearest_num = lst[0]
    nearest_idx = 0
    min_diff = 99999999

    if lst[0] is not None:
        min_diff = abs(num - lst[0])
    
    
    for i in range(1, len(lst) ):
        if lst[i] is  None:
            continue
        diff = abs(num - lst[i])
        if diff <= min_diff:
            nearest_num = lst[i]
            nearest_idx = i
            min_diff = diff
            
    return (nearest_num, nearest_idx)


def rotate_180(angle):
    return (angle + 180) % 360

In [49]:
def find_main_angle(angle , expectation_idx = 0): # expectation_idx is previous angle_idx

    # old mode
    # important_angles = [0 , 30 , 90  , 150 , 180 , 200 , 270 , 340 , 360]
    # angle_names = ["front" , "front-left" , "left" , "back-left" , "back" , "back-right" , "right" , "front-right" , "front"]
    
    # correct mode
    important_angles = [0 , 60 , 90  , 110 , 190 , 240 , 270 , 305 , 350]
    angle_names = ["left" , "front-left" , "front" , "front-right" , "right" , "back-right" , "back" , "back-left" , "left"]


    # temp mode
    # important_angles = [ 270 , 200 , 180 , 150 , 90 , 30 , 0  , 340 , 290]
    # angle_names = [ "right" , "back-right" , "back" , "back-left" , "left" , "front-left" , "front" , "front-right" , "right"]

    nearest = find_nearest(important_angles , angle)
    if nearest is None:
        return "------------" , expectation_idx # nothing detected
    
    ang , idx = nearest
    if 280 > abs(ang - important_angles[expectation_idx]) > 100: # dont mistake on 180 degree
        angle2 = rotate_180(angle)
        ang , idx = find_nearest(important_angles , angle2)
    if idx != expectation_idx + 1: # and  idx != expectation_idx - 1: # dont turn back
        idx = expectation_idx
    return angle_names[idx] , idx
    
    


In [50]:
import os
import random


font = cv2.FONT_HERSHEY_SIMPLEX
x, y = int(video_array[0].shape[1] // 2.5) , video_array[0].shape[0] // 2
color = (150, 35, 200)
font_scale = 2
thickness = 10

r = random.randint(0,12000)

In [51]:
import cv2
import os
import copy
# Set the directory containing the images
li = copy.deepcopy(video_array)

# Create a window to display the video
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

# Loop through each image and display it in the window
c= 0 
expectation_idx = 0
yawObj = Yaw()

yaws = []
main_angles = []

detected_video = []

for img in li:
    if c % shoter_time == 0 :
        yaww = yawObj.predict_yaw(img)
        ang , idx = find_main_angle(yaww , expectation_idx = expectation_idx)
        yaws.append(yaww)
        main_angles.append(ang)
        expectation_idx = idx
    cv2.putText(img, "angle: "+str(ang), (x,y), font, font_scale, color, thickness)
    cv2.imshow('Video', img)
    detected_video.append(img)
    c+=1
    # Wait for 25 milliseconds between each frame (40 frames per second)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the window and close all windows
cv2.destroyAllWindows()

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_14[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [52]:
detected_video = np.array(detected_video)
size = detected_video.shape[2] , detected_video.shape[1]

fps = 30


out = cv2.VideoWriter('sh/mov/RECO/___new7_CORR_'+ name+"_"+ str(r) +'reco.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in detected_video:
    out.write(f)
out.release() 

# light frame-selection

In [22]:
video_array.shape 

((1826, 1080, 1920, 3), 92, 92)

In [26]:
shots = copy.deepcopy(video_array[::shoter_time,:,:])

shots.shape[0] , len(main_angles) , len(yaws)


(261, 261, 261)

In [41]:
yaws

[18.02224093724419,
 17.852824864494096,
 345.8015641434408,
 341.50772405902813,
 16.519957367610232,
 16.551593961384185,
 15.982455438684667,
 15.70863220317591,
 15.864327274750963,
 15.177697503485547,
 13.373656068422633,
 344.13014880778263,
 15.413603784032944,
 15.124951348760021,
 15.834363224148035,
 19.518181820192183,
 19.46350869765803,
 16.463463999011367,
 16.343643655105723,
 16.36944385014698,
 15.73388854421006,
 17.46628553813025,
 15.023692050560555,
 16.567080567090954,
 14.925788083990653,
 14.971014197751174,
 17.54942604553031,
 17.15268668176759,
 17.29331691196785,
 18.228621153225788,
 17.72620250351656,
 16.2663557681434,
 17.304235827034006,
 18.491574138783676,
 19.586205406006457,
 20.169373142277493,
 21.21417133149731,
 21.197462975265864,
 21.873540954264286,
 23.372271543598913,
 23.19992745308207,
 23.882246946553153,
 23.740826975447703,
 25.507351875532436,
 25.83783752688248,
 25.900703891142484,
 26.14060754133095,
 26.078046401258575,
 25.94204

In [9]:

# Path
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir("sh/mov/"+name+"__"+ str(r))

In [14]:
def select_frame_from_period(frames ,angles , angle_range):
    print(angle_range ," -------> ", angles)
    a , b = angle_range
    angle_avg = (a+b) / 2
    angle , idx = find_nearest(angles , angle_avg)
    if angle is  None:
        return None
    if 280 > abs(angle - angle_avg) > 100:
        angle_avg = rotate_180(angle_avg)
        angle , idx = find_nearest(angles , angle_avg)
        angle = rotate_180(angle)
    return frames[idx] , angle





In [15]:
def frame_selection(angles , frames , num_frames = 30):
    selected_frames = []
    selected_angles = []
    angle_period = 360 // num_frames
    frame_period = frames.shape[0] // num_frames
    for i in range(num_frames):
        # print(i)
        half_period = angle_period // 2
        a = i * angle_period - half_period
        b = (i+1) * angle_period - half_period - 1
        c = i * frame_period
        d = (i+1) * frame_period -1
        
        frame_angle = select_frame_from_period( frames[c:d,:,:,:],angles[c:d] , angle_range = (a,b))

        if frame_angle is None:
            continue

        frame , angle = frame_angle
        
        
        selected_frames.append(frame)
        selected_angles.append(angle)

    return selected_frames , selected_angles




        

In [16]:
sel_frames , sel_angles = frame_selection(angles , video_array , 25)

(-7, 6)  ------->  [70.41964977575316, 70.80191340216771, 69.6086814888306, 70.79589856467913, 70.96620939253046, 71.3893491544224, 73.40993754568171, 70.94194684437588, 72.05456200611366]
(7, 20)  ------->  [69.82954597989988, 69.90563175313844, 67.55386956872417, 66.73105047246736, 67.3072800147213, 68.78929328319799, 67.13951087192368, 67.04455157171265, 66.46103848387821]
(21, 34)  ------->  [64.17906719735782, 63.85578825193272, 62.647661825124146, 63.26673651903121, 62.02795469816249, 62.185063832844776, 60.88424909117538, 60.4689615950374, 56.977209214429536]
(35, 48)  ------->  [50.854865577138185, 48.20217966444568, 41.90132704837691, 42.43058664357437, 39.832158065501574, 39.97361816398279, 38.167669397912164, 38.10663651321306, 37.116472172925626]
(49, 62)  ------->  [35.24595760554445, 34.80306919837966, 32.30001570843234, 31.21733300766124, 16.059573668318194, 15.931651009612438, 16.985945726495174, 18.306556444530884, 19.611071816032336]
(63, 76)  ------->  [22.2646080875

In [17]:
sel_angles

[69.6086814888306,
 66.46103848387821,
 56.977209214429536,
 41.90132704837691,
 35.24595760554445,
 22.26460808758054,
 13.095830584316616,
 2.0827794695152875,
 157.57220720723785,
 136.8841697855538,
 134.40568961747948,
 123.26896998612943,
 121.37153934115236,
 245.2606409441492,
 241.72967381881276,
 226.7123634273222,
 223.26959424610942,
 207.34149581788265,
 204.50961470559918,
 181.09095145230003,
 180.7000074527594,
 335.20019554074486,
 320.9734288842045,
 316.8597718158446,
 304.21464748251515]

In [18]:
sel_frames = np.array(sel_frames)
size = sel_frames.shape[2] , sel_frames.shape[1]

fps = 2


out = cv2.VideoWriter('sh/mov/'+ name+"__"+ str(r) +'/reco_.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in sel_frames:
    out.write(f)
out.release() 


In [19]:

# List of images
images = sel_frames # Replace with your list of NumPy arrays

# Write images to disk
for i, image in enumerate(images):
    filename = f"sh/mov/"+ name+"__"+ str(r) +f"/image_{i}.png"
    cv2.imwrite(filename, image)